<a href="https://colab.research.google.com/github/aditidagar/ocular_disease_recog/blob/main/PREPROCESSING/Copy_of_OCULAR_DISEASE_PREPROCESSING_CLAHE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!echo '{"username":"arturusmaximus","key":"4f14194978499e9ae1ad6adb74b94add"}' > /content/kaggle.json
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d andrewmvd/ocular-disease-recognition-odir5k
!unzip ocular-disease-recognition-odir5k.zip
!mv ODIR-5K ODIR-5K_old
!mv ODIR-5K_old/ODIR-5K ODIR-5K
!mkdir ODIR-5K/Validation_Images


Streaming output truncated to the last 5000 lines.
  inflating: preprocessed_images/2179_left.jpg  
  inflating: preprocessed_images/2179_right.jpg  
  inflating: preprocessed_images/217_left.jpg  
  inflating: preprocessed_images/217_right.jpg  
  inflating: preprocessed_images/2180_left.jpg  
  inflating: preprocessed_images/2180_right.jpg  
  inflating: preprocessed_images/2181_left.jpg  
  inflating: preprocessed_images/2181_right.jpg  
  inflating: preprocessed_images/2182_left.jpg  
  inflating: preprocessed_images/2182_right.jpg  
  inflating: preprocessed_images/2183_left.jpg  
  inflating: preprocessed_images/2183_right.jpg  
  inflating: preprocessed_images/2184_left.jpg  
  inflating: preprocessed_images/2184_right.jpg  
  inflating: preprocessed_images/2185_left.jpg  
  inflating: preprocessed_images/2185_right.jpg  
  inflating: preprocessed_images/2187_left.jpg  
  inflating: preprocessed_images/2187_right.jpg  
  inflating: preprocessed_images/2189_left.jpg  
  inflating

In [4]:
import tensorflow as tf
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import IPython.display as display
import numpy as np
import glob
import random

BATCH_SIZE = 32
CLASS_NAMES = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

# Set a random seed so we always use the same validation set (which is randomly sampled from dataset)
random.seed(7)

training_images_file = 'ODIR-5K/Training Images'
testing_images_file = 'ODIR-5K/Testing Images'
labels_file = 'ODIR-5K/data.xlsx'

diseases = {'abnormal pigment ': 'O', 'age-related macular degeneration': 'A', 'anterior segment image': 'DELETE', 'arteriosclerosis': 'O', 'asteroid hyalosis': 'O', 'atrophic change': 'O', 'atrophy': 'O', 'branch retinal artery occlusion': 'O', 'branch retinal vein occlusion': 'O', 'cataract': 'C', 'central retinal artery occlusion': 'O', 'central retinal vein occlusion': 'O', 'central serous chorioretinopathy': 'O', 'chorioretinal atrophy': 'O', 'chorioretinal atrophy with pigmentation proliferation': 'O', 'choroidal nevus': 'NaN', 'congenital choroidal coloboma': 'O', 'depigmentation of the retinal pigment epithelium': 'O', 'diabetic retinopathy': 'D', 'diffuse chorioretinal atrophy': 'O', 'diffuse retinal atrophy': 'O', 'drusen': 'O', 'dry age-related macular degeneration': 'A', 'epiretinal membrane': 'O', 'epiretinal membrane over the macula': 'O', 'fundus laser photocoagulation spots': 'O', 'glaucoma': 'G', 'glial remnants anterior to the optic disc': 'O', 'hypertensive retinopathy': 'H', 'hypertensive retinopathy,diabetic retinopathy': 'D', 'idiopathic choroidal neovascularization': 'O', 'image offset': 'DELETE', 'intraretinal hemorrhage': 'O', 'intraretinal microvascular abnormality': 'O', 'laser spot': 'O', 'lens dust': 'DELETE', 'low image quality': 'DELETE', 'low image quality,maculopathy': 'DELETE', 'macular coloboma': 'O', 'macular epiretinal membrane': 'O', 'macular hole': 'O', 'macular pigmentation disorder': 'NaN', 'maculopathy': 'O', 'mild nonproliferative retinopathy': 'D', 'moderate non proliferative retinopathy': 'D', 'morning glory syndrome': 'O', 'myelinated nerve fibers': 'O', 'myopia retinopathy': 'M', 'myopic maculopathy': 'M', 'myopic retinopathy': 'M', 'no fundus image': 'DELETE', 'normal fundus': 'N', 'old branch retinal vein occlusion': 'O', 'old central retinal vein occlusion': 'O', 'old chorioretinopathy': 'O', 'old choroiditis': 'O', 'optic disc edema': 'O', 'optic discitis': 'O', 'optic disk epiretinal membrane': 'O', 'optic disk photographically invisible': 'DELETE', 'optic nerve atrophy': 'O', 'oval yellow-white atrophy': 'O', 'pathological myopia': 'M', 'peripapillary atrophy': 'O', 'pigment epithelium proliferation': 'O', 'pigmentation disorder': 'O', 'post laser photocoagulation': 'O', 'post retinal laser surgery': 'O', 'proliferative diabetic retinopathy': 'D', 'punctate inner choroidopathy': 'O', 'refractive media opacity': 'O', 'retina fold': 'O', 'retinal artery macroaneurysm': 'O', 'retinal detachment': 'O', 'retinal pigment epithelial hypertrophy': 'O', 'retinal pigment epithelium atrophy': 'O', 'retinal pigmentation': 'O', 'retinal vascular sheathing': 'O', 'retinitis pigmentosa': 'O', 'retinochoroidal coloboma': 'O', 'rhegmatogenous retinal detachment': 'O', 'severe nonproliferative retinopathy': 'D', 'severe proliferative diabetic retinopathy': 'D', 'silicone oil eye': 'O', 'spotted membranous change': 'O', 'suspected abnormal color of  optic disc': 'O', 'suspected cataract': 'C', 'suspected diabetic retinopathy': 'D', 'suspected glaucoma': 'G', 'suspected macular epimacular membrane': 'O', 'suspected microvascular anomalies': 'O', 'suspected moderate non proliferative retinopathy': 'D', 'suspected retinal vascular sheathing': 'O', 'suspected retinitis pigmentosa': 'O', 'suspicious diabetic retinopathy': 'D', 'tessellated fundus': 'O', 'vascular loops': 'O', 'vessel tortuosity': 'O', 'vitreous degeneration': 'O', 'vitreous opacity': 'O', 'wedge white line change': 'O', 'wedge-shaped change': 'O', 'wet age-related macular degeneration': 'A', 'white vessel': 'O'}

#load labels to pandas
labels = pd.read_excel(labels_file, index_col=0) 
print(labels['Left-Diagnostic Keywords'][0])

cataract


Labeling Images by renaming them, removing low quality images 

In [5]:
file_path = 'ODIR-5K/Training Images'
all_paths = []
for element in glob.glob(file_path+"/*.jpg"):
  all_paths.append(element)
paths = []

#adding label information to image names
for u_id in labels.index:
  labelL = ""
  labelR = ""
  diagnosticL = labels['Left-Diagnostic Keywords'][u_id]
  diagnosticR = labels['Right-Diagnostic Keywords'][u_id]
  diagnosticL = diagnosticL.split("，") 
  diagnosticR = diagnosticR.split("，") 
  for d in diagnosticL:
    if d in diseases:
        if labelL != "":
          labelL+="&"+diseases[d]
        else:
          labelL+=diseases[d]
    else:
        labelL+="DELETE"
  for d in diagnosticR:
    if d in diseases:
      if labelR != "":
        labelR+="&"+diseases[d]
      else:
        labelR+=diseases[d]
    else:
      labelR+="DELETE"

  if file_path+"/"+str(labels['Left-Fundus'][u_id]) in all_paths:
    filename = str(labels['Left-Fundus'][u_id]).split(".")
    os.rename(file_path+"/"+str(labels['Left-Fundus'][u_id]), file_path+"/"+filename[0]+"-"+labelL+".jpg")
    paths.append(file_path+"/"+str(labels['Left-Fundus'][u_id])+"/"+labelL)

  if file_path+"/"+str(labels['Right-Fundus'][u_id]) in all_paths:
    filename = str(labels['Right-Fundus'][u_id]).split(".")
    os.rename(file_path+"/"+str(labels['Right-Fundus'][u_id]), file_path+"/"+filename[0]+"-"+labelR+".jpg")
    paths.append(file_path+"/"+str(labels['Right-Fundus'][u_id])+"/"+labelR)

#deleting low quality images    
items_to_remove = []
add_mix_info = []

for element in glob.glob("ODIR-5K/Training Images/*.jpg"):
    img_name = element.split("/")[-1]
    img_label = img_name.split("-")
    if "DELETE" in img_label[-1]:
      items_to_remove.append(element)
    elif "&" in img_label[-1]:
      add_mix_info.append(element)

for e in items_to_remove:
  os.remove(e)

for e in add_mix_info:
  pom = e.split(".")
  os.rename(e, pom[0]+"&X"+".jpg")      

In [6]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

     |████████████████████████████████| 102 kB 4.2 MB/s 
     |████████████████████████████████| 47.6 MB 82 kB/s 
albumentations==1.1.0 is successfully installed


Image Enhancement and Resizing. Creation of Validation Set by Random Selection


In [7]:
import cv2 as cv
import albumentations as A

transform = A.Compose([
   # A.CLAHE(always_apply=True, p=1.0, clip_limit=(1, 4), tile_grid_size=(43, 34))
   A.CLAHE(always_apply=True, p=1.0, clip_limit=(1, 2), tile_grid_size=(12, 6)),
   A.HueSaturationValue(always_apply=True, p=1.0, hue_shift_limit=(0, 0), sat_shift_limit=(0, 14), val_shift_limit=(3, 0))
])

def loadAndCropCenterResizeCV2(img, newSize):
    #img = cv.imread(imgPath)
    width, height, ______ = img.shape
    if width == height:
        return cv.resize(img, newSize)
    length = min(width, height)
    left = (width - length) // 2
    top = (height - length) // 2
    right = (width + length) // 2
    bottom = (height + length) // 2
    return cv.resize(img[left:right, top:bottom, :], newSize)

def clahe_resize(impath):
  img = cv.imread(impath)

  # Convert to RGB Color Space
  image = cv.cvtColor(img, cv.COLOR_BGR2RGB)

  # Apply Albumentations Augmentation (CLAHE)
  transformed = transform(image=image)
  # Get result
  transformed_image = transformed["image"]
  # Convert back to BGR colorspace
  transformed_image_BGR = cv.cvtColor(transformed_image, cv.COLOR_BGR2RGBA)

  #clahe = cv.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))
  #L, a, b = cv.split(cv.cvtColor(img, cv.COLOR_BGR2Lab))
  #cl1 = clahe.apply(L)
  #eq_image = cv.cvtColor(cv.merge([cl1, a, b]), cv.COLOR_Lab2BGR)


  eq_image = loadAndCropCenterResizeCV2(transformed_image_BGR, (250, 250))


  
  cv.imwrite(impath,eq_image)


In [8]:
#images enhancing and resizing (takes a while ~19min)
all_paths = []
for element in glob.glob("ODIR-5K/Training Images/*.jpg"):
  all_paths.append(element)
  clahe_resize(element)

# Set a random seed so we always use the same validation set (which is randomly sampled from dataset)
random.seed(7)

#creating validation set
num_to_select = 1950                          
list_of_random_items = random.sample(sorted(all_paths), num_to_select)
for element in list_of_random_items:
  p = element.split("/")
  os.replace(element, "ODIR-5K/Validation_Images/"+p[-1])

In [9]:
!rm -rf ODIR-5K/Testing\ Images/
!rm -rf ODIR-5K/data.xlsx
!zip -r ODIR-5K_contrast.zip ODIR-5K/

Streaming output truncated to the last 5000 lines.
  adding: ODIR-5K/Training Images/2692_right-N.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/3027_right-N.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/4083_left-D.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/1034_right-N.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/2903_right-N.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/1220_left-G.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/4355_left-N.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/1605_left-M.jpg (deflated 2%)
  adding: ODIR-5K/Training Images/2239_left-C.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/4190_left-D.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/2356_left-N.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/2066_right-H.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/2439_right-N.jpg (deflated 2%)
  adding: ODIR-5K/Training Images/215_left-O.jpg (deflated 1%)
  adding: ODIR-5K/Training Images/3036_right-N.j

In [10]:
from google.colab import drive
drive.mount('/amd/')

Mounted at /amd/


In [11]:
!cp -r ODIR-5K_contrast.zip /amd/My\ Drive/ML490

In [12]:
def show_class_distribution():
  N = 0
  D=0
  G=0
  C=0
  A=0
  H=0
  M=0
  O = 0
  X = 0
  for element in glob.glob("ODIR-5K/Training Images/*.jpg"):
    img_name = element.split("/")[-1]
    img_label = img_name.split("-")
    if img_label[-1] == "N.jpg":
      N += 1
    elif img_label[-1] == "D.jpg":
      D +=1
    elif img_label[-1] == "G.jpg":
      G +=1
    elif img_label[-1] == "C.jpg":
      C +=1
    elif img_label[-1] == "A.jpg":
      A +=1
    elif img_label[-1] == "H.jpg":
      H +=1
    elif img_label[-1] == "M.jpg":
      M +=1
    elif img_label[-1] == "O.jpg":
      O +=1
    elif "&" in img_label[-1]:
      X +=1  
  print(N, D, G, C, A, H, M, O, X)
show_class_distribution()

1982 984 140 178 167 78 170 520 395
